# Ship chase

This example imagines the discovery of a ships log found in Discovery, detailing a stern chase between two ships, with a map from the period covering the chase area. These are all combined using Folium to create an interactive map, allowing a rich visualisation of the situation. 

## The data

#### Discovery examples

There are plenty of example of ships logs and maps in Discovery, two examples of which are below. If you have access to data from Discovery, feel free to use it in place of the examples here. 
- [Ships logs avaiable on Discovery include the Master's log of HMS Endeavour](https://discovery.nationalarchives.gov.uk/details/r/C2976711)
- [Maps avaiable on Discovery include the map of the world by Jodocus Hondius](https://discovery.nationalarchives.gov.uk/details/r/C1343)

#### Example data used here

Here we are going to use fully artificial data for the ships logs, to ensure a clear and simple example. The overlay map we are using is of the English Channel from 1814. This map is available from the [David Rumsey Map Collection, David Rumsey Map Center, Stanford Libraries](https://www.davidrumsey.com/home), available at [https://www.davidrumsey.com/luna/servlet/detail/RUMSEY~8~1~28335~1120764:British-Channel-;JSESSIONID=75ac7b7b-a3a6-440a-b77a-6f0c1cdfb572?title=Search+Results%3A+List_No+equal+to+%271007.015%27&thumbnailViewUrlKey=link.view.search.url&fullTextSearchChecked=&annotSearchChecked=&dateRangeSearchChecked=&showShareIIIFLink=true&helpUrl=https%3A%2F%2Fdocumentation.lunaimaging.com%2Fdisplay%2FV75D%2FLUNA%2BViewer%23LUNAViewer-LUNAViewer&showTip=false&showTipAdvancedSearch=false&advancedSearchUrl=https%3A%2F%2Fdocumentation.lunaimaging.com%2Fdisplay%2FV75D%2FSearching%23Searching-Searching](https://www.davidrumsey.com/luna/servlet/detail/RUMSEY~8~1~28335~1120764:British-Channel-;JSESSIONID=75ac7b7b-a3a6-440a-b77a-6f0c1cdfb572?title=Search+Results%3A+List_No+equal+to+%271007.015%27&thumbnailViewUrlKey=link.view.search.url&fullTextSearchChecked=&annotSearchChecked=&dateRangeSearchChecked=&showShareIIIFLink=true&helpUrl=https%3A%2F%2Fdocumentation.lunaimaging.com%2Fdisplay%2FV75D%2FLUNA%2BViewer%23LUNAViewer-LUNAViewer&showTip=false&showTipAdvancedSearch=false&advancedSearchUrl=https%3A%2F%2Fdocumentation.lunaimaging.com%2Fdisplay%2FV75D%2FSearching%23Searching-Searching), used under a CC license.

## Imports and helper functions

The first step, as always is to import the required set of libraries. As this notebook focusses on being an example of what can be built with Folium, some of the data storage and manipulation is done in a [helper function file](./helper_functions_alt.py). Feel free to have a look at the file if you wish to see the raw data or details of the functions. 

In [ ]:
%pip install -q folium
%pip install -q zipfile
import folium
import folium.plugins as plugins
import zipfile
import urllib.request

import helper_functions as hf

## The base map

To start of, the base map is specified, drawing a larger map than the overlay to ensure there is enough space to show everything.

In [ ]:
chase_map = folium.Map(location=[50.5, -1.5], zoom_start=6)

chase_map

## Highlighting the coastline

With the different overlays, and the potential different projections and accuracy levels of different maps, it is important to ensure that an accurate coastline is highlighted for context. This data is avaiable from [The World Bank](https://datacatalog.worldbank.org/search/dataset/0038272/World%20Bank%20Official%20Boundaries?version=1) in GeoJSON format. 

In [ ]:
data = urllib.request.urlretrieve("https://datacatalogfiles.worldbank.org/ddh-published/0038272/DR0046666/wb_boundaries_geojson_highres.zip?versionId=2023-01-19T09:29:18.1778038Z", "wb_boundaries_geojson_highres.zip")
data_unzip = zipfile.ZipFile("wb_boundaries_geojson_highres.zip", "r")
data_unzip.extractall()

coastline_data = "WB_Boundaries_GeoJSON_highres/WB_Coastlines.geojson"

folium.GeoJson(coastline_data).add_to(chase_map)

chase_map

## Downloading and adding the overlay

Overlaying the map needs a few more values to be specified than the coastline. At this stage, a control to toggle layers on and off is also included. 

In [ ]:
folium.raster_layers.ImageOverlay(
    name="English Channel Map",
    image="./historic_channel_map.jpg",
    bounds=[[48, -8.0], [52.3, 2.8]],
    opacity=0.5,
    interactive=False,
    zindex=1,
).add_to(chase_map)

folium.LayerControl().add_to(chase_map)

chase_map

## Adding the ships paths

With the maps in place, the next step is to add the ships paths. The data is stored as a series of waypoints with timestamps, so this is done as a two-stage process. First, the lists of waypoints are looped though and added to the map as a series of markers. Another loop uses the timestamps and the GeoJson plugin to include lines between each waypoint that come into existence in order and at the correct relative time on an animation, giving a visual indication of each ships relative position during the chase. 

Unfortunately (at time of writing) there is a bug within the timestamped GeoJson plugin, meaning it only accepts dates after 1970. This was an [issue with the base library](https://github.com/socib/Leaflet.TimeDimension/issues/83) that the plugin is built on; the base library has been updated, this just has not (yet) propagated to Folium. Given the visual impact this plugin can provide, it may be worth working around this bug until it is fixed.

In [ ]:
for waypoint in hf.leader_waypoints:
    folium.Marker(
        location=[waypoint["latitude"], waypoint["longitude"]],
        icon=folium.Icon(color="green", icon="info-sign"),
    ).add_to(chase_map)

for waypoint in hf.chaser_waypoints:
    folium.Marker(
        location=[waypoint["latitude"], waypoint["longitude"]],
        icon=folium.Icon(color="red", icon="info-sign"),
    ).add_to(chase_map)

plugins.TimestampedGeoJson(
    {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [waypoint["longitude"], waypoint["latitude"]]
                for waypoint in hf.leader_waypoints
            ]
        },
        "properties": {
            "times": [waypoint["timestamp"] for waypoint in hf.leader_waypoints]
        },
    },
    period="PT1H",
    add_last_point=True,
).add_to(chase_map)

plugins.TimestampedGeoJson(
    {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [waypoint["longitude"], waypoint["latitude"]]
                for waypoint in hf.chaser_waypoints
            ]
        },
        "properties": {
            "times": [waypoint["timestamp"] for waypoint in hf.chaser_waypoints]
        },
    },
    period="PT1H",
    add_last_point=True,
).add_to(chase_map)

chase_map

## How far could the ships see?

Calculating the distance to the horizon from a given height is a simple calculation, needing the height of the observer in meters to give a distance in nautical miles: 

$2.04*\sqrt{h}$, where $h$ is the height of the observer. 

In the helper functions file, this calculation is used to give a viewing range in each of the cardinal directions, which is the plotted onto the map. Note that these visibility diamonds appear as if there is greater visibility in the north/south direction as a side effect of the web-Mercator projection in use. 

For this example, were using heights of 40m and 48m, representing [HMS Surprise](https://en.wikipedia.org/wiki/HMS_Surprise_(replica_ship)) being chased by an imagined larger French Privateer. 

In [ ]:
for waypoint in hf.leader_waypoints:
    additional_waypoints = hf.visibility_around_a_waypoint(waypoint["latitude"], waypoint["longitude"], hf.height_of_surprise_in_m)
    visibility_coords = []
    for additional_waypoint in additional_waypoints:
        visibility_coords.append([additional_waypoint[0], additional_waypoint[1]])
    folium.Polygon(
        locations=visibility_coords,
        color="green",
        fill=True,
        fill_color="green",
        fill_opacity=0.5,
    ).add_to(chase_map)

for waypoint in hf.chaser_waypoints:
    additional_waypoints = hf.visibility_around_a_waypoint(waypoint["latitude"], waypoint["longitude"], hf.height_of_acheron_in_m)
    visibility_coords = []
    for additional_waypoint in additional_waypoints:
        visibility_coords.append([additional_waypoint[0], additional_waypoint[1]])
    folium.Polygon(
        locations=visibility_coords,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.5,
    ).add_to(chase_map)

chase_map

# Conclusion

With a couple of data sources, and some work with Folium, we have created a visualisation showing the relative positions of two ships during a chase, with context provided by an era-relevant map, and information on how far the ships could see. If such data are available, we could build this map up further, with information on the weather, whether the ships had any visibility of each other (visibility to the horizon is not the same as visibility of another ship), locations of any actions, information about the crew, and so on. Lets take a final look at the map, and save it as an HTML file for sharing.

In [ ]:
chase_map.save("chase_map.html")

chase_map